In [1]:
!pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 143.2 MB/s eta 0:00:00a 0:00:01


In [1]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf

2023-07-09 17:58:52.175761: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
%cd ..

/home/jovyan/karim


In [3]:
test_df = pd.read_csv('../data/test.csv')
images_df = pd.read_csv('../data/images.csv')

In [4]:
model_cohort = tf.keras.models.load_model('./logs/new_1/seed4_512_cohort_ep052-loss0.012-val_loss0.018-val_acc0.997-val_f1s0.998.h5',
                                             compile = False)

2023-07-09 17:58:54.575064: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-09 17:58:55.137099: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11550 MB memory:  -> device: 0, name: Tesla K80, pci bus id: 0001:00:00.0, compute capability: 3.7


In [5]:
model_aitype = tf.keras.models.load_model('./logs/new_1/sig_seed4_512_aitype_ep063-loss0.014-val_loss0.225-val_acc0.828-val_f1s0.770.h5',
                                             compile = False)

In [6]:
model_isosynd_all = tf.keras.models.load_model('./logs/new_1/seed4_512_isosynd_all_ep046-loss0.026-val_loss0.195-val_acc0.955-val_f1s0.924.h5',
                                             compile = False)

In [7]:
model_gene = tf.keras.models.load_model('./logs/new_1/4c_seed4_512_gene_ep046-loss0.345-val_loss0.126-val_acc1.000-val_f1s1.000.h5',
                                             compile = False)

In [8]:
classes_cohort = ['Normal', 'Amelogenesis Imperfecta', 'Dentinogenesis Imperfecta']

In [9]:
classes_aitype = ['AI_Hypomature',
                 'AI_Hypoplastic',
                 'AI_Hypocalcification/Hypomineralization',
                 'AI_Hypomature/AI_Hypoplastic/Taurodontism']

In [10]:
classes_isosynd_all = ['None', 'Isolated', 'Syndromic']

In [11]:
classes_gene = ['AMELX', 'ENAM', 'COL17A1', 'COL7A1']

In [ ]:
cohort_pred = []
ai_type_pred = []
Responsible_Gene_Name_pred = []
Is_Isolated_Syndromic_pred = []
patient_id_list = test_df['Patient_ID'].tolist()
for patient_id in patient_id_list:
    all_imgs = images_df.loc[images_df['Patient_ID'] == patient_id]['Image_ID'].tolist()
    # print('nb images: ', len(all_imgs))
    preds_cohort = []
    for img_name in all_imgs:
        img = cv2.imread('../data/images/'+img_name)
        img = img[:,:,0:3]
        img = cv2.resize(img, (512,512))
        img_p = img.astype('float32')
        img_p = img_p/255.
        img_p = np.expand_dims(img_p, 0)
        preds_cohort.append(model_cohort.predict(img_p, verbose=0)[0])
    pred_class = np.argmax(np.mean(np.array(preds_cohort), axis=0))
    cohort_pred.append(classes_cohort[pred_class])
    if pred_class == 0:
        ai_type_pred.append('None')
        Responsible_Gene_Name_pred.append('None')
        Is_Isolated_Syndromic_pred.append('None')
    elif pred_class == 2:
        ai_type_pred.append('None')
        Responsible_Gene_Name_pred.append('None')
        Is_Isolated_Syndromic_pred.append('Isolated')
    elif pred_class == 1:
        preds_aitype = []
        preds_isosynd_all = []
        for img_name in all_imgs:
            img = cv2.imread('../data/images/'+img_name)
            img = img[:,:,0:3]
            img = cv2.resize(img, (512,512))
            img_p = img.astype('float32')
            img_p = img_p/255.
            img_p = np.expand_dims(img_p, 0)
            preds_aitype.append(model_aitype.predict(img_p, verbose=0)[0])
            preds_isosynd_all.append(model_isosynd_all.predict(img_p, verbose=0)[0])
        pred_class_aitype = np.argmax(np.mean(np.array(preds_aitype), axis=0))
        pred_class_isosynd_all = np.argmax(np.mean(np.array(preds_isosynd_all), axis=0))
        ai_type_pred.append(classes_aitype[pred_class_aitype])
        Is_Isolated_Syndromic_pred.append(classes_isosynd_all[pred_class_isosynd_all])
        if pred_class_isosynd_all == 0:
                Responsible_Gene_Name_pred.append('None')
        else:
            if pred_class_aitype == 0:
                if pred_class_isosynd_all == 1:
                    Responsible_Gene_Name_pred.append('MMP20')
                elif pred_class_isosynd_all == 2:
                    Responsible_Gene_Name_pred.append('SLC10A7')
            elif pred_class_aitype == 1:
                if pred_class_isosynd_all == 1:
                    preds_gene = []
                    for img_name in all_imgs:
                        img = cv2.imread('../data/images/'+img_name)
                        img = img[:,:,0:3]
                        img = cv2.resize(img, (512,512))
                        img_p = img.astype('float32')
                        img_p = img_p/255.
                        img_p = np.expand_dims(img_p, 0)
                        preds_gene.append(model_gene.predict(img_p, verbose=0)[0])
                    pred_class_gene = np.argmax(np.mean(np.array(preds_gene), axis=0))
                    Responsible_Gene_Name_pred.append(classes_gene[pred_class_gene])
                elif pred_class_isosynd_all == 2:
                    Responsible_Gene_Name_pred.append('FAM20A')
            elif pred_class_aitype == 2:
                if pred_class_isosynd_all == 1:
                    Responsible_Gene_Name_pred.append('FAM83H')
                elif pred_class_isosynd_all == 2:
                    Responsible_Gene_Name_pred.append('CNNM4')
            elif pred_class_aitype == 3:
                Responsible_Gene_Name_pred.append('DLX3')

2023-07-09 17:59:11.597700: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8500
2023-07-09 17:59:11.899916: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:234] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 3.7
2023-07-09 17:59:11.899953: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:237] Used ptxas at ptxas
2023-07-09 17:59:11.900016: W tensorflow/compiler/xla/stream_executor/gpu/redzone_allocator.cc:318] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


In [ ]:
test_df['Cohort'] = cohort_pred
test_df['AI_Type'] = ai_type_pred
test_df['Responsible_Gene_Name'] = Responsible_Gene_Name_pred
test_df['Is_Isolated_Syndromic'] = Is_Isolated_Syndromic_pred

test_df.to_csv('./submissions/new_1.csv', index=False, encoding='UTF-8')